In [ ]:
import urllib.request

url = 'https://shapenet.cs.stanford.edu/ericyi/shapenetcore_partanno_v0.zip'
urllib.request.urlretrieve(url, 'data.zip')
from zipfile import ZipFile
with ZipFile('data.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('data')
    
url = 'https://github.com/chrdiller/pyTorchChamferDistance/archive/master.zip'
urllib.request.urlretrieve(url, 'chamfer.zip')
with ZipFile('chamfer.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('')
    
url = 'https://github.com/meder411/PyTorch-EMDLoss/archive/master.zip'
urllib.request.urlretrieve(url, 'emd.zip')
with ZipFile('emd.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('')
    
!python setup.py install

In [ ]:
from __future__ import print_function
import argparse
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
from datasets import PartDataset
import torch.nn.functional as F
import torch.cuda as cuda
from pic2points_model import pic2points
from torch.nn.parallel import DataParallel
from torch.autograd import Variable
import torch
from chamfer_distance import ChamferDistance
from emd import EMDLoss

dist =  EMDLoss()

chamferDist = ChamferDistance()

def main():
    manualSeed = random.randint(1, 10000) # fix seed
    print("Random Seed: ", manualSeed)
    random.seed(manualSeed)
    torch.manual_seed(manualSeed)

    dataset = PartDataset(root = 'data/PartAnnotation/', pic2point = True, npoints = 2500)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True, num_workers=8)
    print("number of training data:"+ str(len(dataset)))
    test_dataset = PartDataset(root = 'data/PartAnnotation/', pic2point = True, train = False, npoints = 2500)
    testdataloader = torch.utils.data.DataLoader(test_dataset, batch_size=16,shuffle=True, num_workers=8)
    print("number of testing data:"+ str(len(test_dataset)))

    # creat model
    print("model building...")
    model = pic2points(num_points=2500)
    model.cuda()

    # load pre-existing weights
  #  if opt.model != '':
   #     model.load_state_dict(torch.load(opt.model))

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    num_batch = len(dataset) / 32

    print('training mode ------------------')
    for epoch in range(100):
        print("epoch:"+str(epoch))
        for i, data in enumerate(dataloader, 0):
            im, points = data
            im, points = Variable(im), Variable(points)
            im, points = im.cuda(), points.cuda()
            pred = model(im)
            dist1, dist2 = chamferDist(pred, points)
            loss = (torch.mean(dist1)) + (torch.mean(dist2))
            emd_cost = torch.sum(dist(pred.cuda().double(), points.cuda().double()))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if i % 50 is 0:
                print("training loss is:" + str(loss.item()))

        loss_test = 0
        for i, data in enumerate(dataloader, 0):
            im_test, points_test = data
            im_test, points_test = Variable(im_test), Variable(points_test)
            im_test, points_test = im_test.cuda(), points_test.cuda()
            pred_test = model(im_test)
            dist1, dist2 = chamferDist(pred_test, points_test)
            loss_test = (torch.mean(dist1)) + (torch.mean(dist2))
            emd_test = torch.sum(dist(pred_test.cuda().double(), points_test.cuda().double()))
        print("Testing loss is:" + str(loss_test.item()))

if __name__ == '__main__':
    num_cuda = cuda.device_count()
    print("number of GPUs have been detected:" + str(num_cuda))
    #with torch.cuda.device(1):
    main()

number of GPUs have been detected:1
Random Seed:  2866
number of training data:15990
number of testing data:1785
model building...
training mode ------------------
epoch:0
training loss is:6.619719982147217
training loss is:0.019710224121809006
training loss is:0.010472262278199196
training loss is:0.015419594943523407
training loss is:0.009701407514512539
training loss is:0.014090017415583134
training loss is:0.007824632339179516
training loss is:0.010071864351630211
training loss is:0.009033850394189358
training loss is:0.010414700955152512
training loss is:0.008028747513890266
training loss is:0.005313782021403313
training loss is:0.005707578733563423
training loss is:0.008179374039173126
training loss is:0.007255700416862965
training loss is:0.007073071785271168
training loss is:0.008052784949541092
training loss is:0.006962824612855911
training loss is:0.006793557666242123
training loss is:0.004782738164067268
Testing loss is:0.004579806700348854
epoch:1
training loss is:0.0049696